In [12]:
import plotly.offline as py
import pandas as pd

def country2code(t):
    df_2 = pd.read_csv(
        'reference/country2code.csv',
        names=['country','country_code'])  #reference/country2code.csv是转换器文件
    dict_country2code = df_2.set_index(
                        'country').T.to_dict('list')  #country2code的dataframe轉化為字典
    return dict_country2code[t.split(', ')[-1]][0]  #将所有记者的country转化为code

def merge(df_merge_1, filename):
    df_merged = pd.read_csv(  #这是完整的country表
        'reference/code&country.csv',names=['country_code','country']
        ).join(df_merge_1, on='country_code').fillna(0)  #将数据合并到完整的country表上，填充NaN
    df_merged.to_csv(filename)
    return df_merged

def paint_world_map(title,locations,z,text):
    data = [ dict(
            type = 'choropleth',
            locations = locations,  #國家代碼，可使用上面的方法把國家名轉化為代碼
            z = z,  #數據
            text = text,  #國家名，此處因為使用了dataframe的數據，因此所有項目都是一一對應關係
            colorscale = [[0,"rgb(0, 30, 155)"],[0.1,"rgb(15, 45, 165)"],[0.2,"rgb(30, 60, 175)"],
                          [0.3,"rgb(45, 75, 185)"],[0.4,"rgb(60, 90, 195)"],[0.5,"rgb(75, 105, 205)"],
                          [0.6,"rgb(90, 120, 215)"],[0.7,"rgb(115, 145, 225)"],[0.8,"rgb(140, 170, 235)"],
                          [0.9,"rgb(165, 195, 245)"],[1,"rgb(255, 255, 255)"]],  #深淺變化
            autocolorscale = False,  #深淺變化不符合審美，建議False
            reversescale = True,  #此項根據上面colorscale，設為True
            marker = dict(  #設置國界線
                line = dict (
                    color = 'rgb(180,180,180)',
                    width = 0.5
                ) ),
            colorbar = dict(
                autotick = True,
                title = 'amount'),
          ) ]

    layout = dict(
        title = 'Global Data Journalist Distribution<br>Source:\
                <a href="http://jplusplus.github.io/global-directory/">\
                Global Data Journalist Directory</a>',  #設置標題
        geo = dict(
            showframe = False,  #設置有無地圖大邊框
            showcoastlines = False,  #設置有無海岸線
            projection = dict(
                type = 'Mercator'
            )
        )
    )

    fig = dict( data=data, layout=layout )
    return py.iplot( fig, validate=False, filename='d3-world-map' )


df_gh = pd.read_csv(
    'reference/journalists-data-including-github-contributions.csv'
    )  #geopy-raw.csv是上一cell中导出的记者地址原始数据
df_0d = df_gh.reindex(
    ['name','institution','city','github'],axis=1)

df_0d['country_code'] = df_gh['city'].apply(country2code)  #将所有记者的country转化为code
df_code_country_ratio = pd.DataFrame()

SyntaxError: invalid syntax (<ipython-input-12-7ce8fb417ec8>, line 1)

In [9]:
df_code_amount = pd.DataFrame(
    df_0d['country_code'].value_counts()
    ).rename(columns={'country_code':'data'})
    #根據country2code，country轉化為code

df_code_country_data = merge(
    df_code_amount,
    'journalist_country_amount')

locations = list(df_code_country_data['country_code'])
text = list(df_code_country_data['country'])
z = list(df_code_country_data['data'])
title = 'Global Data Journalist Distribution<br>Source:\
                <a href="http://jplusplus.github.io/global-directory/">\
                Global Data Journalist Directory</a>'

paint_world_map(title,locations,z,text)

In [10]:
import numpy as np
df_0d['contribution_sum'] = df_gh.iloc[:,4:].sum(axis = 1).astype("int")
df_contribution_sum = pd.pivot_table(
    df_0d,index=['country_code'],values=['contribution_sum'],aggfunc=np.sum
    ).rename(columns={'contribution_sum':'data'})#通过pivot_table获得对应国家的contribution


df_code_country_data = merge(
    df_contribution_sum,
    'journalist_country_amount')

locations = list(df_code_country_data['country_code'])
text = list(df_code_country_data['country'])
z = list(df_code_country_data['data'])
title = 'Global Data Journalist Distribution<br>Source:\
                <a href="http://jplusplus.github.io/global-directory/">\
                Global Data Journalist Directory</a>'

paint_world_map(title,locations,z,text)

In [11]:
df_code_country_data = pd.DataFrame(
    df_code_amount['data']).rename(
    columns={'data':'country_code'}
    ).join(df_contribution_sum['data'])


# [i for i in df_code_country_data['country_code']]/[i for i in df_code_country_data['country_code']]
df_code_country_data['ratio'] = df_code_country_data.apply(lambda x: x[1] / x[0], axis=1)
df_code_country_data
locations = list(df_code_country_data.index)
# text = list(df_code_country_data['ratio'])
z = list(df_code_country_data['ratio'])
title = 'Global Data Journalist Distribution<br>Source:\
                <a href="http://jplusplus.github.io/global-directory/">\
                Global Data Journalist Directory</a>'

paint_world_map(title,locations,z,text)

In [ ]:
df_code_country_data = merge(
    pd.DataFrame(  #根據country2code，country轉化為code
        df_gh['country'].value_counts()
        ).rename(columns={'country':'data'}),
    'countribution_country_amount'
    )

locations = list(df_code_country_data['code'])
text = list(df_code_country_data['country'])
z = list(df_code_country_data['data'])
title = 'Global Data Journalist Distribution<br>Source:\
                <a href="http://jplusplus.github.io/global-directory/">\
                Global Data Journalist Directory</a>'

paint_world_map(title,locations,z,text)

In [45]:
import pandas as pd
# from geopy.geocoders import Nominatim
# import time
df_gh = pd.read_csv(
    'reference/journalists-data-including-github-contributions.csv'
    )
df_2 = pd.read_csv('reference/country2code.csv'
                   ,names=['COUNTRY','CODE']) 
list_country = [i.split(', ')[-1] for i in df_gh['city']]
dict_country2code = df_2.set_index('COUNTRY').T.to_dict('list')  #country2code的dataframe轉化為字典
list_code = [dict_country2code[i][0] for i in list_country]  #将所有记者的country转化为code
df_gh['country'] = list_code
df_merge_1 = pd.DataFrame(df_gh['country'].value_counts()).rename(columns={'country':'data'})
df_merge_2 = pd.read_csv(
    'reference/code&country.csv',names=['code','country']
    )  #这是完整的country表
df_code_country_data = df_merge_2.join(
    df_merge_1, on='code'
    ).fillna(0)  #将数据合并到完整的country表上，填充NaN
df_code_country_data

,code,country,data
0,AFG,Afghanistan,0.0
1,ALB,Albania,0.0
2,DZA,Algeria,0.0
3,ASM,American Samoa,0.0
4,AND,Andorra,0.0
5,AGO,Angola,0.0
6,AIA,Anguilla,0.0
7,ATG,Antigua and Barbuda,0.0
8,ARG,Argentina,3.0
9,ARM,Armenia,0.0


In [ ]:
from datetime import datetime
from dateutil import parser

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import time
list_raw=[]
geolocator = Nominatim(user_agent="my-application")
df_dj = pd.read_csv("reference/journalists-data.csv",
            names=['name','agency','location','github'])
for i in df_dj['location']:
    location=geolocator.geocode(i, language='en',timeout=60)
    list_raw.append(location.raw)
    time.sleep(2)
df = pd.DataFrame(list_raw)
df.to_csv('geopy-raw.csv')

In [30]:
import plotly.plotly as py
import pandas as pd

df_1 = pd.read_csv('geopy-raw.csv')  #geopy-raw.csv是上一cell中导出的记者地址原始数据
df_2 = pd.read_csv('reference/country2code.csv'
                   ,names=['COUNTRY','CODE'])  #reference/country2code.csv是转换器文件
list_country = [i.split(', ')[-1] for i in df_1['display_name']]  #獲取地址中的country
dict_country2code = df_2.set_index('COUNTRY').T.to_dict('list')  #country2code的dataframe轉化為字典
list_code = [dict_country2code[i][0] for i in list_country]  #将所有记者的country转化为code
dict_count = {}
for m in list_code:  #计数code
    dict_count[m] = list_code.count(m) #根據country2code，country轉化為code
df_merge_1 = pd.DataFrame(dict_count, index=['data']).T
#因为计数结果未包含所有country，所以将计数结果转化为dataframe，准备合并到完整的country表上
df_merge_2 = pd.read_csv('reference/code&country.csv',names=['code','country'])  #这是完整的country表
df_code_country_data = df_merge_2.join(df_merge_1, on='code').fillna(0)  #将数据合并到完整的country表上，填充NaN
df_merge_1
df_merge_0

,country
USA,46
DEU,23
FRA,22
ITA,19
ESP,14
NLD,11
CHE,8
GBR,6
CAN,6
BRA,5


In [7]:
import plotly.plotly as py
import pandas as pd

df_selected_jobs = pd.read_csv("reference/0 jobs.csv")
list_state_code=[]
dict_count={}
for i in df_selected_jobs["Location"]:
    try: 
        list_state_code.append(i.split(', ')[1][:2])
    except:
        pass
for i in range(len(list_state_code)):
    list_state_code[i]=list_state_code[i].upper()
    if list_state_code[i]=='D.':
        list_state_code[i]='DC'
for i in list_state_code:
    dict_count[i]=list_state_code.count(i)
code=list(dict_count.keys())
value=list(dict_count.values())

# for col in df.columns:
#     df[col] = df[col].astype(str)

    
    
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = code,
        z = value,
        locationmode = 'USA-states',
        text = code,
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "how many positions")
        ) ]

layout = dict(
        title = 'U.S journalism jobs',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )